# Домашняя работа

Сделайте mapper и reducer, чтобы посчитать среднее и дисперсию оценок за фильм.

 # Подготовка окружения

In [102]:
import opendatasets as od
import pandas as pd
import time
from pathlib import Path

In [2]:
dataset_path = Path('imdb-user-reviews', 'song_lyrics.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews')

Dataset URL: https://www.kaggle.com/datasets/sadmadlad/imdb-user-reviews


100%|██████████| 15.1M/15.1M [00:01<00:00, 12.5MB/s]


Реализация через цикл (предпогаем, что мы не знаем сколько у нас фильмов):

In [154]:
import json


n, mean, M2 = 0, 0.0, 0
for path in Path('imdb-user-reviews').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        with open(path, 'r') as f:
            info = json.load(f)
        score = float(info['movieIMDbRating'])
        n += 1
        delta = score - mean
        mean += delta / n
        M2 += delta * (score - mean)
       
print(n, mean, (M2 / n) ** (1/2))

10 8.03 1.0517128885774865


На основе этого кода соберите mapper и reducer:

In [155]:

file_list=[]
for path in Path('imdb-user-reviews').glob('**/*'):
    if path.is_file() and path.suffix == '.json':
        file_list.append(path)

def mapper(path):
    with open(path, 'r') as f:
        info = json.load(f)
    score = float(info['movieIMDbRating'])
    return (1,score,0)
    
    

def reducer(score_data1, score_data2):
    n, mean, M2=score_data1
    n += 1
    delta = score_data2[1] - mean
    mean += delta / n
    M2 += delta * (score_data2[1] - mean)
    return n, mean, M2

In [156]:
%%time
from functools import reduce
n, mean, M2 = reduce(reducer, map(mapper,file_list))
print(n, mean, (M2 / n) ** (1/2))

10 8.03 1.0517128885774865
CPU times: total: 0 ns
Wall time: 2.01 ms
